In [2]:
import pandas as pd
import json
from datetime import datetime

In [6]:

df = pd.read_csv(r'D:\DoAnMayHoc\score_regression\data\annonimized.csv')
df.rename(columns={
    "concat('it001',`assignment_id`)": 'assignment_id',
    "concat('it001',`problem_id`)": 'problem_id',
    "concat('it001', username)": 'username',
    "concat('it001',`language_id`)": 'language_id',
}, inplace=True)

df.drop(columns=['coefficient', 'language_id'], inplace=True)

df.to_csv('data_cleaned.csv', index=False)